## Лабораторна робота №2 з дисципліни "Машинне навчання"
### Виконав: Студент групи КМ-12 Пєшков Антон

**Порядок виконання завдання:**
1. Відкрити та зчитати дані з наданого файлу.
2. Визначити збалансованість набору даних. Вивести кількість об’єктів кожного класу.

In [1]:
import pandas as pd
from math import sqrt, nan
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score

df = pd.read_csv('KM-12-2.csv')
display(df)
df.GT.value_counts()

,GT,Model_1_0,Model_1_1,Model_2_0,Model_2_1
0,1,0.133871,0.866129,0.265398,0.734602
1,1,0.707424,0.292576,0.894910,0.105090
2,1,0.355263,0.644737,0.957991,0.042009
3,1,0.674354,0.325646,0.060504,0.939496
4,1,0.674354,0.325646,0.991170,0.008830
...,...,...,...,...,...
4995,0,0.417549,0.582451,0.983547,0.016453
4996,0,0.828717,0.171283,0.010689,0.989311
4997,0,0.707424,0.292576,0.974684,0.025316
4998,0,0.840336,0.159664,0.950068,0.049932


GT
1    2500
0    2500
Name: count, dtype: int64

3. Для зчитаного набору даних виконати наступні дії: 
    1. Обчислити всі метрики:
        - Accuracy;
        - Precision;
        - Recall;
        - F-Score;
        - Matthews Correlation Coefficient;
        - Balanced Accuracy;
        - Youden’s J statistics;
        - Area Under Curve for Precision-Recall Curve;
        - Area Under Curve for Receiver Operation Curve,

        для кожної моделі при різних значеннях порогу класифікатора (крок зміни порогу обрати самостійно). 
    2. Збудувати на одному графіку в одній координатній системі (величина порогу; значення метрики) графіки усіх обчислених метрик, відмітивши певним чином максимальне значення кожної з них.
    3. Збудувати в координатах (значення оцінки класифікаторів; кількість об’єктів кожного класу) окремі для кожного класу графіки кількості об’єктів та відмітити вертикальними лініями оптимальні пороги відсічення для кожної метрики.
    4. Збудувати для кожного класифікатору $PR$-криву та $ROC$-криву, показавши графічно на них значення оптимального порогу.

In [2]:
class Plotter:
    def __init__(self, df: pd.DataFrame, col_true: str, col_model_0: str, col_model_1: str):
        self.df = df
        self.col_true = df[col_true]
        self.model0 = df[col_model_0]
        self.model1 = df[col_model_1]

    def basic_metrics(self, p: float):
        TP = (self.col_true & (self.model1 > p)).sum()
        TN = ((~self.col_true) & (self.model1 <= p)).sum()
        FP = ((~self.col_true) & (self.model1 > p)).sum()
        FN = (self.col_true & (self.model1 <= p)).sum()
        return TP, TN, FP, FN

    def metrics_showdown(self, p: float, beta: float = 1):
        assert 0 <= p <= 1
        TP, TN, FP, FN = self.basic_metrics(p)

        ACCURACY = (TP + TN)/(TP + TN + FP + FN)
        PRECISION = TP/(TP + FP)
        RECALL_0 = TN/(TN + FP)
        RECALL_1 = TP/(TP + FN)
        F_SCORE = (1 + beta**2)*PRECISION*RECALL_1/(PRECISION + RECALL_1)
        MCC = nan \
            if 0 == (denom := (TP + FP)*(TP + FN)*(TN + FP)*(TN + FN)) \
            else (TP*TN - FP*FN)/sqrt(denom)

        BALANCED_ACCURACY = (RECALL_0 + RECALL_1)/2
        J_STATS = RECALL_0 + RECALL_1 - 1
        print(f"{TP = }, {TN = }, {FP = }, {FN = }")
        print(f"{ACCURACY = :.4f}, {PRECISION = :.4f}, recall = {RECALL_1:.4f}")
        print(f"{F_SCORE = :.4f}, {BALANCED_ACCURACY = :.4f}, {J_STATS = :.4f}")
        print(f"{MCC = :.4f}")

    def calculate_auc_prc(self, p: float):
        pass
        # precision, recall, thresholds = precision_recall_curve(self.col_true, self.model1 > p)

    def curves(self):
        pass

In [3]:
N = 4
models =[('Model_1_0', 'Model_1_1'), ('Model_2_0', 'Model_2_1')]

for i, model in enumerate(models, 1):
    pl = Plotter(df, 'GT', *model)

    for step in range(N):
        p = step/N
        print(f"Для порогу {p = :.2f} класифікатора #{i} маємо:")
        pl.metrics_showdown(p)
        print()


Для порогу p = 0.00 класифікатора #1 маємо:
TP = 2500, TN = 0, FP = 2500, FN = 0
ACCURACY = 0.5000, PRECISION = 0.5000, recall = 1.0000
F_SCORE = 0.6667, BALANCED_ACCURACY = 0.5000, J_STATS = 0.0000
MCC = nan

Для порогу p = 0.25 класифікатора #1 маємо:
TP = 2303, TN = 1145, FP = 1355, FN = 197
ACCURACY = 0.6896, PRECISION = 0.6296, recall = 0.9212
F_SCORE = 0.7480, BALANCED_ACCURACY = 0.6896, J_STATS = 0.3792
MCC = 0.4279

Для порогу p = 0.50 класифікатора #1 маємо:
TP = 1844, TN = 2020, FP = 480, FN = 656
ACCURACY = 0.7728, PRECISION = 0.7935, recall = 0.7376
F_SCORE = 0.7645, BALANCED_ACCURACY = 0.7728, J_STATS = 0.5456
MCC = 0.5470

Для порогу p = 0.75 класифікатора #1 маємо:
TP = 1258, TN = 2412, FP = 88, FN = 1242
ACCURACY = 0.7340, PRECISION = 0.9346, recall = 0.5032
F_SCORE = 0.6542, BALANCED_ACCURACY = 0.7340, J_STATS = 0.4680
MCC = 0.5276

Для порогу p = 0.00 класифікатора #2 маємо:
TP = 2500, TN = 0, FP = 2500, FN = 0
ACCURACY = 0.5000, PRECISION = 0.5000, recall = 1.0000
F_

4. Зробити висновки щодо якості моделей, визначити кращу модель.
5. Створити новий набір даних, прибравши з початкового  набору $ (50 + 5К)\% $ об’єктів класу 1, вибраних випадковим чином. Параметр К представляє собою залишок від ділення дня народження студента на дев’ять та має визначатися в програмі на основі дати народження студента, яка задана в програмі у вигляді текстової змінної формату ‘DD-MM’. 
6. Вивести відсоток видалених об’єктів класу 1 та кількість елементів кожного класу після видалення. 
7. Виконати дії п.3 для нового набору даних.
8. Визначити кращу модель.
9. Пояснити вплив незбалансованості набору даних на прийняте рішення.

In [4]:
pass